In [1]:
#getting and working with data
import pandas as pd
import numpy as np
import re
import os
import datetime as dt
import string
import scipy

#visualizing results
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context("poster")
sns.set_style("ticks")

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 15000)
pd.set_option('display.max_colwidth', -1)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

/Users/abbieschindler/opt/anaconda3/envs/analysis/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


### Get data paths

In [5]:
#create list of dir paths

orig_path = '/Users/abbieschindler/Documents/Schindler_Lab/Data/Anesthetized/MFB'

dir_path_list = os.listdir(orig_path)

data_dir_paths = []
for directory in dir_path_list:
    int_path = orig_path + '/' + directory
    data_dir_paths.append(int_path)
    
data_dir_paths

['/Users/abbieschindler/Documents/Schindler_Lab/Data/Anesthetized/MFB/200803_MFB_blast_1m_CD56',
 '/Users/abbieschindler/Documents/Schindler_Lab/Data/Anesthetized/MFB/200731_MFB_sham_1m_CD53.2',
 '/Users/abbieschindler/Documents/Schindler_Lab/Data/Anesthetized/MFB/.DS_Store',
 '/Users/abbieschindler/Documents/Schindler_Lab/Data/Anesthetized/MFB/200804_MFB_blast_1m_CD57',
 '/Users/abbieschindler/Documents/Schindler_Lab/Data/Anesthetized/MFB/200915_MFB_sham_1m_SA226.1',
 '/Users/abbieschindler/Documents/Schindler_Lab/Data/Anesthetized/MFB/201216_MFB_blast_1m_SA287.2',
 '/Users/abbieschindler/Documents/Schindler_Lab/Data/Anesthetized/MFB/181117_MFB_sham_1m_practice1',
 '/Users/abbieschindler/Documents/Schindler_Lab/Data/Anesthetized/MFB/200804_MFB_sham_1m_CD54.1',
 '/Users/abbieschindler/Documents/Schindler_Lab/Data/Anesthetized/MFB/200917_MFB_blast_1m_SA234.3',
 '/Users/abbieschindler/Documents/Schindler_Lab/Data/Anesthetized/MFB/201214_MFB_blast_1m_SA.287.1']

In [8]:
#create list of file paths 

data_folder_names = ['/Hz/BATCH_PC/STACKED_PC/Stacked_Current', 
                     '/tonicphasic/BATCH_PC/STACKED_PC/Stacked_Current', 
                     '/uA/BATCH_PC/STACKED_PC/Stacked_Current']

data_paths = []

for name in data_folder_names:
    print(name)
    for directory in data_dir_paths:
        print(directory)
        int_path = str(directory + name)
        data_paths.append(int_path)
    
data_paths[0]

/Hz/BATCH_PC/STACKED_PC/Stacked_Current
/Users/abbieschindler/Documents/Schindler_Lab/Data/Anesthetized/MFB/200803_MFB_blast_1m_CD56
/Users/abbieschindler/Documents/Schindler_Lab/Data/Anesthetized/MFB/200731_MFB_sham_1m_CD53.2
/Users/abbieschindler/Documents/Schindler_Lab/Data/Anesthetized/MFB/.DS_Store
/Users/abbieschindler/Documents/Schindler_Lab/Data/Anesthetized/MFB/200804_MFB_blast_1m_CD57
/Users/abbieschindler/Documents/Schindler_Lab/Data/Anesthetized/MFB/200915_MFB_sham_1m_SA226.1
/Users/abbieschindler/Documents/Schindler_Lab/Data/Anesthetized/MFB/201216_MFB_blast_1m_SA287.2
/Users/abbieschindler/Documents/Schindler_Lab/Data/Anesthetized/MFB/181117_MFB_sham_1m_practice1
/Users/abbieschindler/Documents/Schindler_Lab/Data/Anesthetized/MFB/200804_MFB_sham_1m_CD54.1
/Users/abbieschindler/Documents/Schindler_Lab/Data/Anesthetized/MFB/200917_MFB_blast_1m_SA234.3
/Users/abbieschindler/Documents/Schindler_Lab/Data/Anesthetized/MFB/201214_MFB_blast_1m_SA.287.1
/tonicphasic/BATCH_PC/STACK

'/Users/abbieschindler/Documents/Schindler_Lab/Data/Anesthetized/MFB/200803_MFB_blast_1m_CD56/Hz/BATCH_PC/STACKED_PC/Stacked_Current'

In [14]:
Hz_cols = ['time', '05', '10', '20', '40', '60']
tp_cols = ['time', 'tonic1', 'phasic1', 'tonic2', 'phasic2']
uA_cols = ['time', '25', '50', '100', '150', '200', '300']

data_Hz = pd.DataFrame()
data_tonicphasic = pd.DataFrame()
data_uA = pd.DataFrame()

for file in data_paths:
    
    if file.split('/')[-5] == '.DS_Store':
        continue
    
    try:
        data_int = pd.read_table(file, header=None)
        data_int = pd.DataFrame(data = data_int)
    
        #get max value for each stim param
        max_vals = data_int[(data_int[0] > 4.9) & (data_int[0] < 8.0)].max(axis=0).values
    
        #get stim type from path    
        stim_param = file.split('/')[-4]
    
        if stim_param == 'Hz':
            #create df with max values
            data_int_Hz = pd.DataFrame(data=max_vals).T
            data_int_Hz.columns = Hz_cols
            #fill meta data
            data_int_Hz['stim_param'] = file.split('/')[-4]
            data_int_Hz['animal'] = file.split('/')[-5].split('_')[4]
            data_int_Hz['date'] = file.split('/')[-5].split('_')[0]
            data_int_Hz['stim_loc'] = file.split('/')[-5].split('_')[1]
            data_int_Hz['group'] = file.split('/')[-5].split('_')[2]
            data_int_Hz['tp'] = file.split('/')[-5].split('_')[3]
            #add to larger df
            if data_Hz.shape[0] == 0:
                data_Hz = data_int_Hz
            else:
                data_Hz = pd.concat([data_Hz, data_int_Hz], axis=0)
    
        if stim_param == 'tonicphasic':
            #create df with max values
            data_int_tp = pd.DataFrame(data=max_vals).T
            data_int_tp.columns = tp_cols
            #fill meta data
            data_int_tp['stim_param'] = file.split('/')[-4]
            data_int_tp['animal'] = file.split('/')[-5].split('_')[4]
            data_int_tp['date'] = file.split('/')[-5].split('_')[0]
            data_int_tp['stim_loc'] = file.split('/')[-5].split('_')[1]
            data_int_tp['group'] = file.split('/')[-5].split('_')[2]
            data_int_tp['tp'] = file.split('/')[-5].split('_')[3]
            #add to larger df
            if data_tonicphasic.shape[0] == 0:
                data_tonicphasic = data_int_tp
            else:
                data_tonicphasic = pd.concat([data_tonicphasic, data_int_tp], axis=0)
            
            
        if stim_param == 'uA':
            #create df with max values
            data_int_uA = pd.DataFrame(data=max_vals).T
            data_int_uA.columns = uA_cols
            #fill meta data
            data_int_uA['stim_param'] = file.split('/')[-4]
            data_int_uA['animal'] = file.split('/')[-5].split('_')[4]
            data_int_uA['date'] = file.split('/')[-5].split('_')[0]
            data_int_uA['stim_loc'] = file.split('/')[-5].split('_')[1]
            data_int_uA['group'] = file.split('/')[-5].split('_')[2]
            data_int_uA['tp'] = file.split('/')[-5].split('_')[3]
            #add to larger df
            if data_uA.shape[0] == 0:
                data_uA = data_int_uA
            else:
                data_uA = pd.concat([data_uA, data_int_uA], axis=0)
                
    except:
        print(file)
    
    
    
    
data_Hz

/Users/abbieschindler/Documents/Schindler_Lab/Data/Anesthetized/MFB/200804_MFB_blast_1m_CD57/tonicphasic/BATCH_PC/STACKED_PC/Stacked_Current
/Users/abbieschindler/Documents/Schindler_Lab/Data/Anesthetized/MFB/201216_MFB_blast_1m_SA287.2/tonicphasic/BATCH_PC/STACKED_PC/Stacked_Current
/Users/abbieschindler/Documents/Schindler_Lab/Data/Anesthetized/MFB/181117_MFB_sham_1m_practice1/tonicphasic/BATCH_PC/STACKED_PC/Stacked_Current
/Users/abbieschindler/Documents/Schindler_Lab/Data/Anesthetized/MFB/200917_MFB_blast_1m_SA234.3/tonicphasic/BATCH_PC/STACKED_PC/Stacked_Current
/Users/abbieschindler/Documents/Schindler_Lab/Data/Anesthetized/MFB/200804_MFB_blast_1m_CD57/uA/BATCH_PC/STACKED_PC/Stacked_Current


,time,05,10,20,40,60,stim_param,animal,date,stim_loc,group,tp
0,7.9,-0.111,25.596,-0.111,25.596,-0.111,Hz,CD56,200803,MFB,blast,1m
0,7.9,0.502,0.636,3.102,15.267,17.136,Hz,CD53.2,200731,MFB,sham,1m
0,7.9,25.596,-0.111,25.596,-0.111,25.596,Hz,CD57,200804,MFB,blast,1m
0,7.9,0.278,0.557,3.908,16.587,17.155,Hz,SA226.1,200915,MFB,sham,1m
0,7.9,2.268,4.748,23.966,30.821,23.530,Hz,SA287.2,201216,MFB,blast,1m
0,7.9,0.796,1.587,4.346,16.272,24.107,Hz,practice1,181117,MFB,sham,1m
0,7.9,25.596,-0.111,25.596,-0.111,25.596,Hz,CD54.1,200804,MFB,sham,1m
0,7.9,-0.352,27.987,-0.352,27.987,-0.352,Hz,SA234.3,200917,MFB,blast,1m
0,7.9,1.134,1.804,12.109,29.617,29.414,Hz,SA.287.1,201214,MFB,blast,1m


In [21]:
data_Hz.to_csv('data_Hz.csv')
data_tonicphasic.to_csv('data_tonicphasic.csv')
data_uA.to_csv('data_uA.csv')

In [20]:
remove = ['CD56', 'CD57', 'CD54.1', 'SA234.3']
data_tonicphasic[~data_tonicphasic['animal'].isin(remove)].groupby('group').mean()

,time,tonic1,phasic1,tonic2,phasic2
group,,,,,
blast,7.9,1.1130,4.179,0.924,4.389
sham,7.9,0.3335,1.681,0.162,3.010


In [ ]:
data_Hz[(data_Hz['time'] > 4.9) & (data_Hz['time'] < 8.0)].max(axis=0).values[1:]

In [ ]:
Hz_cols = ['time', '05', '10', '20', '40', '60']

data = pd.read_table(data_paths[1], header=None)
df = pd.DataFrame(data = data)
df.columns = Hz_cols
print(df.shape)

df.max = df[(df['time'] > 4.9) & (df['time'] < 8.0)] .max(axis=0)
df.head()

In [ ]:
data_4h = df[df['TP'] == '4h']
data_4h_blood = data_4h[data_4h['Sample'] == 'blood']
data_4h_blood = data_4h_blood[data_4h_blood['Ship'] == 2]
print(data_4h_blood.shape)
data_4h_brain = data_4h[data_4h['Sample'] == 'brain']
print(data_4h_brain.shape)

data_6m = df[df['TP'] == '6m']
data_6m_blood = data_6m[data_6m['Sample'] == 'blood']
print(data_6m_blood.shape)
data_6m_brain = data_6m[data_6m['Sample'] == 'brain']
print(data_6m_brain.shape)

### First look

In [ ]:
#viz 

for param in df.columns.values:
    try:
        print(param)
        #viz
        g = sns.catplot(x='TP', y=param, data=df, kind='bar', height=5, aspect=1, ci=68, hue='Group', 
                        col='Sample', sharey=False)
        plt.show()
        
        print('\n')
        
    except:
        pass

### Determine sig diff cytokines

In [ ]:
cytokines = ['IL-1α', 'IL-1β',
       'IL-2', 'IL-4', 'IL-5', 'IL-6', 'IL-7', 'IL-9', 'IL-10',
       'IL-12(p40)', 'IL-12(p70)', 'IL-13', 'IL-15', 'IL-17', 'G-CSF',
       'GM-CSF', 'INFγ', 'IP-10', 'MKC', 'MCP-1', 'MIP-1α', 'MIP-1β',
       'MIP-2', 'RANTES', 'TNFα']

data = data_4h_blood

ttest_dict = {}

for cytokine in cytokines:
    print(cytokine)
    try:
        #viz
        g = sns.catplot(x='Group', y=cytokine, data=data, kind='bar', height=3, aspect=1, ci=68)
        plt.show()
        
        print('\n')
        
    except:
        pass
    
    try:
        x=data[data['Group'] == 1][cytokine].dropna().values
        y=data[data['Group'] == 2][cytokine].dropna().values
        ttest = scipy.stats.ttest_ind(x, y)
        print(ttest[1], '\n')
        
        #save to dict only if p value is below 0.1
        if ttest[1] < 0.1:
            ttest_dict[cytokine] = ttest

    except:
        print('can not run stats')

### corr and heat map

In [ ]:
sig_cytokines = ttest_dict.keys()
data1 = data[data['Group'] == 1]
data2 = data[data['Group'] == 2]

fig, ax = plt.subplots(figsize=(13,13))
corr1 = data1[sig_cytokines].corr()
mask = np.triu(np.ones_like(corr1, dtype=bool))
ax = sns.heatmap(corr1, annot=True, vmin=-1, vmax=1, center=0, cmap = 'coolwarm', mask=mask,
                square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.yticks(rotation = 0)
plt.show()

fig, ax = plt.subplots(figsize=(13,13))
corr2 = data2[sig_cytokines].corr()
mask = np.triu(np.ones_like(corr2, dtype=bool))
ax = sns.heatmap(corr2, annot=True, vmin=-1, vmax=1, center=0, cmap = 'coolwarm', mask=mask,
                square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.yticks(rotation = 0)
plt.show()

### Z score and heat map

In [ ]:
cytokines = ['IL-1α', 'IL-1β',
       'IL-2', 'IL-4', 'IL-5', 'IL-6', 'IL-7', 'IL-9', 'IL-10',
       'IL-12(p40)', 'IL-12(p70)', 'IL-13', 'IL-15', 'IL-17', 'G-CSF',
       'GM-CSF', 'INFγ', 'IP-10', 'MKC', 'MCP-1', 'MIP-1α', 'MIP-1β',
       'MIP-2', 'RANTES', 'TNFα']

data_4h_blood_z = pd.DataFrame(data=scipy.stats.zscore(data_4h_blood[cytokines], axis=0, nan_policy='omit'), columns=cytokines)
data_4h_blood_z['Group'] = data_4h_blood['Group'].values
data_4h_blood_z.head()

from sklearn.preprocessing import StandardScaler
#scale data algo
scaler = StandardScaler()
ss = scaler.fit_transform(data_4h_blood[cytokines])
data_4h_blood_ss = pd.DataFrame(data=ss, columns=cytokines)
data_4h_blood_ss['Group'] = data_4h_blood['Group'].values
data_4h_blood_ss.head()

In [ ]:
df_melt = pd.melt(data_4h_blood_z, id_vars=['Group'], 
                             value_vars=['IL-1α', 'IL-1β',
       'IL-2', 'IL-4', 'IL-5', 'IL-6', 'IL-7', 'IL-9', 'IL-10',
       'IL-12(p40)', 'IL-12(p70)', 'IL-13', 'IL-15', 'IL-17', 'G-CSF',
       'GM-CSF', 'INFγ', 'IP-10', 'MKC', 'MCP-1', 'MIP-1α', 'MIP-1β',
       'MIP-2', 'RANTES', 'TNFα']).dropna()

df_melt['log_value'] = np.log10(df_melt['value'])

df_melt.head()

In [ ]:
groupby = df_melt.groupby(['Group', 'variable'])['value'].mean().reset_index()
#viz
groupby = groupby.pivot('Group', 'variable', "value")
plt.figure(figsize=(15,3))
ax = sns.heatmap(groupby, cmap="coolwarm")
plt.show()

### Cluster and heat map

In [ ]:
data_4h_blood.set_index(['Group', 'Animal']).drop(['Ship', 'Sample', 'TP'])
#.dropna(axis=1)

In [ ]:
from sklearn.preprocessing import StandardScaler
data_4h_blood.set_index(['Group', 'Ship', 'Sample', 'Animal', 'TP']).dropna(axis=1)

sns.clustermap(data_4h_blood.set_index(['Group', 'Ship', 'Sample', 'Animal', 'TP']).dropna(axis=1),
               metric="euclidean",  method="ward", z_score=1,
               annot=True, vmin=-1, vmax=1, center=0, cmap = 'coolwarm', 
               square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
data_4h_blood_z

In [ ]:
corr = dep_no_outliers.groupby(["Group"]).corr()
fig, ax = plt.subplots(figsize=(20, 20))
sns.heatmap(corr, annot=True)

We will use IQR (interquartile range) to determine outliers within each group. We will use the definition of outlier as any data point more than 1.5 IQRs below the first quartile or above the third quartile.

In [ ]:
#create new data frame organized by group so we can compute outliers for each group individually
data_groups = df.unstack(level = -1)
data_groups.head()

In [ ]:
#compute quartiles, IQRs, and bounds for each parameter for each group
quartile_1 = data_groups.quantile(0.25)
quartile_3 = data_groups.quantile(0.75)
iqr = quartile_3 - quartile_1
lower_bound = quartile_1 - (iqr * 1.5)
upper_bound = quartile_3 + (iqr * 1.5)
lower_bound.head()

In [ ]:
#use bounds to exclude any data points outside of the bounds (outliers will be replaced with NaN)
outliers = data_groups[(data_groups <= upper_bound) & (data_groups >= lower_bound)]
#stack to return dataframe to original orientation
dep_no_outliers = outliers.stack()

In [ ]:
df_describe = pd.DataFrame(data = dep_no_outliers.groupby(["Group"]).describe())
df_describe